In [15]:
%load_ext autoreload
%autoreload 2
import numpy as np
import qutip as qt 
from CD_control_optimization.batch_optimizer import BatchOptimizer
from CD_control_optimization.optimization_analysis import OptimizationAnalysis, OptimizationSweepsAnalysis
from CD_control_optimization.optimization_sweeps import OptimizationSweeps
import matplotlib.pyplot as plt
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 3204841177992257769, name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 7996401344554530608
 physical_device_desc: "device: XLA_CPU device"]

In [25]:
N = 50
N_blocks = 6
fock = 2
N_multistart = 10
epochs = 100
use_displacements = False
no_CD_end = False
initial_states = [qt.tensor(qt.basis(2,0),qt.basis(N,0))]
#target_states = [qt.tensor(qt.basis(2,0), c.zero_logical)]
target_states = [qt.tensor(qt.basis(2,0),qt.basis(N,fock))]
#target_states = [qt.tensor(qt.basis(2,0), (qt.coherent(N,alpha) + qt.coherent(N,-alpha)).unit())]
desired_epsilon = 1e-2
term_fid = 1-desired_epsilon
dfid_stop = 1e-8
beta_scale = 2.0
alpha_scale = 0.5
name='Fock %d' % fock

In [27]:
opt = BatchOptimizer(initial_states=initial_states, target_states=target_states, N_blocks=N_blocks,term_fid = term_fid,
N_multistart=N_multistart, use_displacements=use_displacements, no_CD_end=no_CD_end, dfid_stop=dfid_stop, name=name, epochs=epochs, beta_scale=beta_scale, alpha_scale=alpha_scale,
fock=fock)

In [28]:
opt.print_info()

optimization_type: state transfer
N_multistart: 10
N_blocks: 6
term_fid: 0.99
dfid_stop: 1e-08
no_CD_end: False
name: Fock 2
learning_rate: 0.01
epoch_size: 10
epochs: 100
beta_scale: 2.0
alpha_scale: 0.5
use_displacements: False
comment: 
fock: 2
N_cav: 50
filename: Fock 2.h5

Best circuit parameters found:
betas:         [ 0.32683+0.38342j  0.16149-0.28408j  0.95206-1.19131j  0.99119-0.54829j
 -0.69833-1.39427j  0.04657-0.15172j]
alphas:        [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
phis (deg):    [  0.      -40.30846  15.52573  40.95721 -76.13783 -97.55591]
thetas (deg):  [-177.65497  -20.9262  -152.78381  -76.96154    6.64609  -63.83931]
Max Fidelity:  0.296803




In [29]:
opt.optimize()

Start time: 2020-10-22 03:58:16 PM
 Epoch: 25 / 100 Max Fid: 0.991581 Avg Fid: 0.878947 Max dFid: 0.042479 Avg dFid: 0.010056 Elapsed time: 0:00:24.205669 Remaing time: 0:01:12.617007

 Optimization stopped. Term fidelity reached.

optimization_type: state transfer
N_multistart: 10
N_blocks: 6
term_fid: 0.99
dfid_stop: 1e-08
no_CD_end: False
name: Fock 2
learning_rate: 0.01
epoch_size: 10
epochs: 100
beta_scale: 2.0
alpha_scale: 0.5
use_displacements: False
comment: 
fock: 2
N_cav: 50
filename: Fock 2.h5

Best circuit parameters found:
betas:         [ 0.48662-1.46329j  0.74576+0.25115j -0.61522-1.44321j -0.6692 +0.23319j
  0.15176+0.36616j -0.01763+0.20302j]
alphas:        [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
phis (deg):    [  0.      -80.05749  66.21063 -56.98937 -98.56276  -3.9132 ]
thetas (deg):  [ -92.26157  227.5424    80.49157 -103.55416  112.35981   75.83174]
Max Fidelity:  0.991642


all data saved as: Fock 2.h5
termination reason: term_fid
optimization timestamp (start

'2020-10-22 03:58:16 PM'

In [30]:
opt.print_info()

optimization_type: state transfer
N_multistart: 10
N_blocks: 6
term_fid: 0.99
dfid_stop: 1e-08
no_CD_end: False
name: Fock 2
learning_rate: 0.01
epoch_size: 10
epochs: 100
beta_scale: 2.0
alpha_scale: 0.5
use_displacements: False
comment: 
fock: 2
N_cav: 50
filename: Fock 2.h5

Best circuit parameters found:
betas:         [ 0.48662-1.46329j  0.74576+0.25115j -0.61522-1.44321j -0.6692 +0.23319j
  0.15176+0.36616j -0.01763+0.20302j]
alphas:        [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
phis (deg):    [  0.      -80.05749  66.21063 -56.98937 -98.56276  -3.9132 ]
thetas (deg):  [ -92.26157  227.5424    80.49157 -103.55416  112.35981   75.83174]
Max Fidelity:  0.991642




In [31]:
filename = "Fock 2.h5"
ana = OptimizationAnalysis(filename)

In [32]:
ana.plot_fidelities()

RuntimeError: In set_text: Could not load glyph (error code 0x55)

In [ ]:
sweeper = OptimizationSweeps(opt_object=opt)

In [ ]:
sweeper.N_blocks_sweep(min_N_blocks=2, max_N_blocks=10, terminate=True)

In [ ]:
#filename = 'Cat Creation.h5'
#filename = "C:\\Users\\Alec\\Desktop\\Fock 1.h5"
filename = "Fock 3.h5"
sweep_analysis = OptimizationSweepsAnalysis(filename=filename)

In [ ]:
sweep_analysis.sweep_names

In [ ]:
sweep_analysis._load_data()

In [ ]:
sweep_analysis.plot_sweep_fidelities()

In [ ]:
s = timestamps[0]
if len(timestamps) > 1:
    for t in timestamps:
        s =s + ',' + t
print(s)

In [ ]:
a = 'c:/a/test.h5'

In [ ]:
a.split(".")

In [ ]:
opt.optimize()

In [ ]:
c = opt.best_circuit()

In [ ]:
c['betas']

In [ ]:
opt.best_fidelity()

In [ ]:
opt.print_info()

In [ ]:
opt.modify_parameters(N_blocks = 10)

In [ ]:
tf.is_tensor(qt.create(10))

In [ ]:
#filename = 'fock 3.h5'
analysis = OptimizationAnalysis(filename=filename)
#bs = analysis.betas()

In [ ]:
analysis.timestamps

In [ ]:
analysis.data[analysis.timestamps[-1]]['betas'][-1].shape

In [ ]:
analysis.print_info()

In [ ]:
analysis.plot_fidelities()

In [ ]:
analysis.plot_mag_betas()

In [ ]:
analysis.plot_phase_betas()

In [ ]:
analysis.plot_mag_alphas()

In [ ]:
analysis.plot_phase_alphas()

In [ ]:
analysis.

In [ ]:
analysis.plot_average_magnitude_alpha()

In [ ]:
analysis.plot_average_magnitude_beta()

In [ ]:
analysis.data[analysis.timestamps[-1]]

In [ ]:
plt.figure(figsize=(8,6))
if len(all_fids.shape)>1:
    for fids in all_fids:
        plt.semilogy(1-fids)
else:
    plt.semilogy(1-all_fids)
plt.xlabel('epoch')
plt.ylabel('infidelity')
plt.title("100x batch optimization")

In [ ]:
opt.print_info()

In [ ]:
betas, alphas, phis, thetas = opt.get_numpy_vars()

In [ ]:
betas.T.shape

In [ ]:
fids = opt.batch_state_fidelities(opt.betas_rho, opt.betas_angle, opt.alphas_rho, opt.alphas_angle, opt.phis, opt.thetas)

In [ ]:
max_idx = tf.argmax(fids)[0,0].numpy()
max_idx

In [ ]:
fids[max_idx]

In [ ]:
betas[max_idx]

In [ ]:
with tf.device("XLA_GPU:0"):
    a = tf.constant(1.0)

In [ ]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

In [ ]:
tf.config.list_physical_devices()

In [ ]:
chi_ab = 2*np.pi*1e-9*10

In [ ]:
chi_ab

In [ ]:
nbar = 1/(4*chi_ab*1000)
nbar

In [ ]:
np.sqrt(nbar)

In [ ]:
np.sqrt(400.0)

In [ ]:
import h5py

In [ ]:
data = np.array([1.0,2.0,3.0])
with h5py.File('test.h5', "w") as f:
    fidelities = f.create_dataset("fidelities", data = data, maxshape=(None,))

In [ ]:
with h5py.File('test.h5', "a") as f:
    f['fidelities'].resize(4, axis=0)
    f['fidelities'][-1] = 4.0

In [ ]:
fs_MHz = np.linspace(0,0.2,201)
omegas_MRadps = 2*np.pi*fs_MHz
tau_us = 10
argument = omegas_MRadps*tau_us/2/np.pi
y = np.sinc(argument)**2


In [ ]:
plt.figure(figsize=(4,3),dpi=200)
plt.plot(fs_MHz*1e3, y, label='tau=10us')
plt.xlabel('KHz')
plt.ylabel('F')
plt.legend(frameon=False)